In [6]:
from pathlib import Path
from collections import defaultdict
import yaml
import re
import string
import math
import pandas as pd
import duckdb
from unidecode import unidecode
import iteround
from utils.pandas_setup import pandas_setup
pandas_setup()

with open('../folders.yaml', 'r') as in_file:
    folders = yaml.safe_load(in_file)
snapshot_duckdb = folders.get('snapshot_duckdb')
working_duckdb = folders.get("working_duckdb")
temp_duckdb = folders.get("temp_duckdb")
print(f'{snapshot_duckdb = } {Path(snapshot_duckdb).exists() = }')
print(f'{working_duckdb = } {Path(working_duckdb).exists() = }')
print(f'{temp_duckdb = } {Path(temp_duckdb).exists() = }')

project = folders.get("project")
print(f'{project = }')
PUNC = string.punctuation
print(f'{PUNC = }')

type_types = ['erratum', 'retraction', 'letter', 'libguides', 'editorial', 'article', 'preprint',
          'report', 'review', 'standard', 'other', 'reference-entry', 'book', 'dissertation', 
          'paratext', 'supplementary-materials', 'book-chapter', 'dataset', 'grant', 'peer-review']

def reduce_works_topics(lst: list) -> str:
    # print(type(lst), lst)
    if not isinstance(lst, list) or not lst:
        return None
    return '; '.join([item.get('id')  for k, item in enumerate(lst) if k < 3]) # and item.get('score') > 0.5]) # keep top three topics if score > 0.5

def clean_name(s) -> str:
    if s is None or len(s) < 1:
        return s
    return re.sub(' +', ' ',  ''.join([c if c not in PUNC else ' ' for c in unidecode(s).lower()])).strip().capitalize()

with duckdb.connect(f'{working_duckdb}/{project}.duckdb') as db:
    topic_for_map = db.sql(f"SELECT * FROM {project}.topic_to_for").df()
    print(f'{topic_for_map.shape}\n{topic_for_map.head()}')
    dd = dict(zip(topic_for_map.topic_id, topic_for_map.for_weights))
    dd = {k: {kk:vv for kk, vv in v.items() if vv/max(v.values()) > 0.1} for k, v in dd.items()}
    dd = {k:v for k, v in dd.items() if v}
    print(len(dd.keys()), dd)

def transform_topics_to_for(lst: list) -> list[dict]:
    # print(f'{type(lst) = } {lst = }')
    if not lst:
        return {}
    d = defaultdict(float)
    for item in lst:
        score = item['score']
        if not (temp := dd.get(item['id'])):
            return {}
        for k, v in temp.items():
            d[k] += v*score
    d = {k: 10*v/sum(d.values()) for k, v in d.items()}
    # print(f'{d = }')
    scaled = [{'topic_id':k, 'topic_score': v} for k, v in iteround.saferound(d, 0).items() if v > 0.1]
    # print(f'{scaled = }')
    return scaled

ModuleNotFoundError: No module named 'pyarrow'

In [ ]:
class SetupDatabases:

    def __init__(self):
        self.project = project
        self._extract_windows()
        self._set_database()
        df = self.db.sql("SHOW ALL TABLES").df()
        print(df.head(999))
        print("SETUP IS COMPLETED")
        return

    def _extract_windows(self):
        # time windows for citation census and target yeats
        with open('../time_windows.yaml', 'r') as in_file:
            time_windows = yaml.safe_load(in_file)
        self.start_census, self.end_census = time_windows['census']
        self.start_target, self.end_target = time_windows['target']        
        print(f'{self.start_census = } {self.end_census = } {self.start_target = } {self.end_target = }')         
        return
    
    def _set_database(self):
        # open a "memory" database
        self.db = duckdb.connect(":memory: AS memory")
        try:
            self.db.create_function('clean_name', 
                                    clean_name, 
                                    return_type=duckdb.duckdb.typing.DuckDBPyType(str),
                                    null_handling='special',
                                    exception_handling='return_null')
        except Exception as e:
            print(f'{e = }')
        try:
            self.db.create_function('transform_topics_to_for', 
                                    transform_topics_to_for, 
                                    return_type=duckdb.duckdb.typing.DuckDBPyType(list[dict[str,float]]),
                                    null_handling='special',
                                    # exception_handling='return_null'
                                    )
        except Exception as e:
            print(f'{e = }')
        self.db.sql(f"""                
                    SET memory_limit = '48GB';
                    SET threads = 20;
                    -- SET partitioned_write_flush_threshold = 1024;
                    SET preserve_insertion_order = false;                                        
                    SET enable_progress_bar = true;
                    SET temp_directory = '{temp_duckdb}';
                    """)
        self._attach_project_database()
        self._attach_oa_database()
        # self.db.sql("SHOW ALL TABLES").show()
        return self
       
    def _attach_project_database(self):
        # attach the project "ERA" working database 
        db_file = f'{working_duckdb}/{self.project}.duckdb'
        print(f'{db_file = } {Path(db_file).exists() = }')
        self.db.sql(f"ATTACH IF NOT EXISTS '{db_file}'")    # def setup_test(self):
    #     # self._extract_test_topics()
    #     self._apply_test_for()
    def _attach_oa_database(self):
        # attach full OA databases

        self.db.sql("ATTACH IF NOT EXISTS'/home/lc/m/openalex_dec24/duckdb/sources.duckdb'")
        # self.db.sql("SELECT * FROM sources.sources").show()
        self.db.sql("ATTACH IF NOT EXISTS '/home/lc/m/openalex_dec24/duckdb/topics.duckdb'")
        # self.db.sql("SELECT * FROM topics.topics").show() 
        self.db.sql("ATTACH IF NOT EXISTS '/home/lc/m/openalex_dec24/duckdb/works.duckdb'")
        # self.db.sql("SELECT * FROM works.topics").show()                 
        # self.db.sql("SHOW ALL TABLES").show()

        return self
    
    def clear_old_project_tables(self):
        for t in ['authors', 'centiles', 'era', 'era_journal_list', 'sources_era', 'works', 'works_author_count']:
            self.db.sql(f"DROP TABLE IF EXISTS {self.project}.{t}")
        # self.db.sql("SHOW ALL TABLES").show()
        return

In [ ]:
def main():
    return

In [ ]:
if __name__ == "__main__":
    main()
    print("DONE!")